In [ ]:
!pip install -q rank_bm25
!pip -q install -U sentence-transformers
!pip -q install deep_translator
!pip install -q transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1 peft==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00


In [ ]:
#Inference
import torch
import numpy as np
import pandas as pd
from context_retrieval.context_retrieval_vimedqa import *

from tqdm import tqdm
from collections import Counter
from datasets import load_dataset

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import TextClassificationPipeline

In [ ]:
def id_labeling(num_opts):
  option_dict = {2: ['a', 'b'],
               3: ['a', 'b', 'c','ab', 'ac', 'bc','abc'],
               4: ['a', 'b', 'c', 'd', 'ab', 'ac', 'ad', 'bc', 'bd', 'cd','abc', 'abd', 'acd', 'bcd','abcd'],
               5: ['a', 'b', 'c', 'd', 'e', 'ab', 'ac', 'ad', 'ae', 'bc', 'bd', 'be', 'cd', 'ce', 'de', 'abc', 'abd', 'abe', 'acd',
                   'ace', 'ade', 'bcd', 'bce', 'bde', 'cde', 'abcd', 'abce', 'abde', 'acde', 'bcde','abcde']}
  if num_opts in [2,3,4,5]:
    label_list = option_dict[num_opts]
    id2label = {}
    label2id = {}
    for idx, label in enumerate(label_list):
      id2label[idx] = label
      label2id[label] = idx
    num_labels = len(id2label)
    return id2label, label2id, num_labels

def predict_per_sample(df, model, tokenizer):
  BERT_CLS = "[CLS]"
  BERT_BOS = ""
  BERT_SEP = "[SEP]"
  BERT_EOS = ""
  convert_to_binary = {'a': "10000", 'b':"01000", 'c':"00100", 'd':"00010", 'e':"00001",
                       'ab': "11000", 'ac':"10100", 'ad':"10010", 'ae':"10001",
                       'bc':"01100", 'bd':"01010", 'be':"01001", 'cd':"00110", 'ce':"00101", 'de':"00011",
                       'abc': "11100", 'abd':"11010", 'abe':"11001", 'acd':"10110", 'ace':"10101", 'ade':"10011",
                       'bcd': "01110", 'bce':"01101", 'bde':"01011", 'cde':"00111",
                       'abcd':"11110", 'abce':"11101", 'abde':"11011", 'acde':"10111", 'bcde':"01111",'abcde':"11111"}
  id2label, label2id, num_labels = id_labeling(5)
  answers = []
  for key, d in df.iterrows():
    if str(d["context"]) == "nan":
      d["context"] = "None"

    opt_lst = [op for op in d[['option_1', 'option_2', 'option_3', 'option_4', 'option_5', 'option_6']].tolist() if str(op) != '']
    bert_ctx = BERT_CLS + " " + d["question"] + f" {BERT_SEP} " + f" {BERT_SEP} ".join(opt_lst) + " " + BERT_SEP + d["context"] + BERT_EOS

    # Prediction
    res = pipeline(bert_ctx, truncation=True, max_length=model.config.max_position_embeddings)
    lb = res[0][0]["label"].split("_")[-1]
    pred = id2label[int(lb)]
    answer = str(convert_to_binary[pred])
    #answer = convert_to_binary[prediction.item()]
    answer = answer[:len(opt_lst)-1]
    answers.append('{}'.format(answer))
  return answers

def prediction(test_df_path, model, tokenizer):
  test_data = test_set_preprocessing(test_df_path)
  corpus_path = './modified_dataset/ViMed_corpus_dict.csv'
  corpus_df = corpus_preprocessing(corpus_path)
  test_w_context_data = bm25_searcher(test_data, corpus_df, top_k=5)
  #test_w_context_data.to_csv("vi_context_public_test.csv", index = False)
  en_test_w_context_data = translate_df(test_w_context_data)
  #en_test_w_context_data.to_csv("en_context_public_test.csv")
  return predict_per_sample(en_test_w_context_data, model, tokenizer)

In [ ]:
path_model = ".model__transformer_classifier/scibert_scivocab_uncased-finetuned-MedMCQA&FrenchMCQA-v7/checkpoint-30000"
tokenizer = AutoTokenizer.from_pretrained(path_model)
model = AutoModelForSequenceClassification.from_pretrained(path_model)
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=1, device='cpu')

test_df_path = "./modified_dataset/public_test.csv"

answers = prediction(test_df_path, model, tokenizer)
answers

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
6it [02:00, 20.14s/it]


['001', '001', '001', '0', '010', '011']